In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/2023_P1/subjectivity_mining/final/data

/content/drive/MyDrive/2023_P1/subjectivity_mining/final/data


In [ ]:
pwd

'/content/drive/MyDrive/2023_P1/subjectivity_mining/final/data'

In [ ]:
pip install simpletransformers

In [ ]:
import pandas as pd
import tensorflow as tf
import simpletransformers
import torch
import sklearn
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Load the datasets

# Comment this cell out. We will load the dataset everytime to avoid confusion

# test_df = pd.read_csv('olid-test.csv')
# out_df = pd.read_csv('hasoc-train.csv')

# Clean the datasets
# del train_df['id']
# del test_df['id']
# train_df.dropna(axis='index', inplace = True)
# test_df.dropna(axis='index', inplace = True)

# Train 5fold CV models; get a output train_df_st_cross

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel,ClassificationArgs

# 5FCV output for ensemble


def output_df(model,train):
  rskf = StratifiedKFold(n_splits = 5, shuffle = True)
  pred = []
  probabilities = []
  gold = []
  index = []
  for train_index, test_index in rskf.split(train['text'], train['labels']):
    train_df = train.iloc[train_index]
    test_df = train.iloc[test_index]
    model.train_model(train_df)
    predictions, prob = model.predict(test_df.text.to_list())

    gold.extend(test_df['labels'])
    pred.extend(predictions)
    probabilities.extend(softmax(prob, axis = 1)[:,1])
    index.extend(test_index)

  output = pd.DataFrame(columns = ['label','probabilities','predicted', 'ind'])
  output.label, output.probabilities, output.predicted, output.ind = gold,probabilities, pred, index
  output['id'] = [train.id.to_list()[idx] for idx in index]
  output = output[['id','label','probabilities','predicted']]
  return output

train_df = pd.read_csv('hasoc-train.csv')

# get the output table for bert with seed 95489322
model_bert_st = ClassificationModel("bert", "bert-base-cased", args={'reprocess_input_data': True, 'manual_seed':95489322, 'num_train_epochs': 1,
                                                          'overwrite_output_dir': True, 'output_dir':'models/5fold/bert'}, use_cuda = True)
bert_output_st = output_df(model_bert_st, train_df)

# get the output table for hatebert with seed 42
model_hatebert_st = ClassificationModel("bert", "GroNLP/hateBERT",
                                     args={'reprocess_input_data': True, 'manual_seed':42, 'num_train_epochs': 1,
                                         'overwrite_output_dir': True, 'output_dir':'models/5fold/hatebert'}, use_cuda = True)

hatebert_output_st = output_df(model_hatebert_st, train_df)

# get the output table for fbert with seed 31415926
model_fbert_st = ClassificationModel("bert", "diptanu/fBERT", args={'reprocess_input_data': True, 'manual_seed':31415926, 'num_train_epochs': 1,
                                  'overwrite_output_dir': True, 'output_dir':'models/5fold/hatebert'}, use_cuda = True)
fbert_output_st = output_df(model_fbert_st, train_df)

bert_output_st.rename(columns = {'probabilities': 'bert_probabilities', 'predicted':'bert_predicted'}, inplace = True)
hatebert_output_st.rename(columns = {'probabilities': 'hatebert_probabilities', 'predicted':'hatebert_predicted', 'label':'hatebert_label'},inplace = True)
fbert_output_st.rename(columns = {'probabilities': 'fbert_probabilities', 'predicted':'fbert_predicted', 'label':'fbert_label'},inplace = True)

train_df_st = bert_output_st.merge(hatebert_output_st, on = 'id').merge(fbert_output_st, on = 'id')

# check if we merge the tables correctly
print(train_df_st['label'].equals(train_df_st['fbert_label']))
print(train_df_st['label'].equals(train_df_st['hatebert_label']))

# now we can drop the labels
train_df_st.drop(columns = ['fbert_label','hatebert_label'], inplace = True)
train_df_st.to_csv('train_df_st_cross.csv')

# Train 3 basic models with the entire dataset

In [ ]:
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationModel,ClassificationArgs
# Config
# https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

# Train
# https://simpletransformers.ai/docs/classification-models/#training-a-classification-model

# Evaluate
# https://simpletransformers.ai/docs/classification-models/#evaluating-a-classification-model

train_df = pd.read_csv('hasoc-train.csv')
del train_df['id']
train_df.dropna(axis='index', inplace = True)

############### BERT


model_args = ClassificationArgs()
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 1
model_args.manual_seed = 95489322
model_args.output_dir = 'models/mcross/basic_model'
bert_model = ClassificationModel('bert','bert-base-cased', args=model_args)

bert_model.train_model(train_df)

############### HateBERT
hbert_model = ClassificationModel('bert','GroNLP/hateBERT')
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.manual_seed = 42
model_args.output_dir = 'models/mcross/hatebert_model'
model_args.overwrite_output_dir = True

hbert_model = ClassificationModel("bert", "GroNLP/hateBERT", args=model_args)
hbert_model.train_model(train_df)

################ fBERT
fbert_model = ClassificationModel('bert','diptanu/fBERT')
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.manual_seed = 31415926
model_args.output_dir = 'models/mcross/fbert_model'
model_args.overwrite_output_dir = True

fbert_model = ClassificationModel("bert", "diptanu/fBERT", args=model_args)
fbert_model.train_model(train_df)


test_df = pd.read_csv('olid-test.csv')
del test_df['id']
bert_pred, bert_raw_pred = bert_model.predict(list(test_df['text']))
hbert_pred, hbert_raw_pred = hbert_model.predict(list(test_df['text']))
fbert_pred, fbert_raw_pred = fbert_model.predict(list(test_df['text']))

test_df_st = pd.DataFrame()
test_df_st['bert_predicted'] = bert_pred.tolist()
test_df_st['hatebert_predicted'] = hbert_pred.tolist()
test_df_st['fbert_predicted'] = fbert_pred.tolist()
test_df_st['labels'] = test_df['labels']

# test_df_st['bert_raw_pred'] = softmax(bert_raw_pred, axis = 1)[:,1].tolist()
# test_df_st['hbert_raw_pred'] = softmax(hbert_raw_pred, axis = 1)[:,1].tolist()
# test_df_st['fbert_raw_pred'] = softmax(fbert_raw_pred, axis = 1)[:,1].tolist()

X_test = test_df_st[['bert_predicted', 'hatebert_predicted','fbert_predicted']]
Y_test = test_df_st['labels']

test_df_st.to_csv('test_df_st_from_3_basic_models_cross.csv')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

# Feature engineering + Train meta model

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
train_df_st_csv = pd.read_csv('train_df_st_cross.csv').drop(columns = ['Unnamed: 0'])

X_train = train_df_st_csv[['bert_predicted', 'hatebert_predicted', 'fbert_predicted','id']]
Y_train = train_df_st_csv['label']

train_df = pd.read_csv('hasoc-train.csv').drop(columns = 'labels')
X_train = X_train.merge(train_df, on = 'id')

# define feature engineering function
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer




#base_lexicon = pd.read_csv('baseLexicon.txt', sep = '\t', header = None)
#base_lexicon.columns = ['Token_Pos', 'label']

#base_lexicon[['token','pos']] = base_lexicon['Token_Pos'].str.split('_', expand = True)
#abusive =  base_lexicon[base_lexicon['label'] == True]['token']

# def count_base_lexicon(text):
#   lemmatizer = WordNetLemmatizer()
#   text = word_tokenize(text.lower())
#   count = 0
#   for word in text:
#     word = lemmatizer.lemmatize(word)
#     if word in list(abusive):
#       count = count + 1
#   return count

def get_vader(text):
  analyzer = SentimentIntensityAnalyzer()
  score = analyzer.polarity_scores(text)['compound']
  return score

def count_character_length(text):
  count = len(text)
  return count

def count_text_length(text):
  count = len(text.split())
  return count


#X_train['base_lexicon_count'] = X_train['text'].apply(count_base_lexicon)
X_train['character_length_count'] = X_train['text'].apply(count_character_length)
X_train['word_count'] = X_train['text'].apply(count_text_length)
X_train['vader'] = X_train['text'].apply(get_vader)

X_train = X_train[['bert_predicted', 'hatebert_predicted', 'fbert_predicted','character_length_count','word_count','vader']]

from sklearn.preprocessing import RobustScaler
#transformer1 = RobustScaler().fit(X_train[['base_lexicon_count']])
transformer2 = RobustScaler().fit(X_train[['character_length_count']])
transformer3 = RobustScaler().fit(X_train[['word_count']])

# Robust scaler

#X_train['base_lexicon_count'] = transformer1.transform(X_train[['base_lexicon_count']])
X_train['character_length_count'] = transformer2.transform(X_train[['character_length_count']])
X_train['word_count'] = transformer3.transform(X_train[['word_count']])


meta_model = GradientBoostingClassifier(n_estimators=1000,learning_rate=0.1, max_depth=3, random_state=0)
meta_model.fit(X_train, Y_train)


#from sklearn.svm import SVC
#meta_model = SVC()
#meta_model.fit(X_train, Y_train)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
<ipython-input-8-b95b90dc8f3d>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['character_length_count'] = transformer2.transform(X_train[['character_length_count']])
<ipython-input-8-b95b90dc8f3d>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['word_coun

GradientBoostingClassifier(n_estimators=1000, random_state=0)

# Apply features to test_df and predict on test_df

In [ ]:
# meta_model_prediction_X_test = meta_model.predict(X_test)
# from sklearn.metrics import classification_report
# print(classification_report(test_df_st['labels'],meta_model_prediction_X_test))

              precision    recall  f1-score   support

           0       0.87      0.92      0.90       620
           1       0.76      0.66      0.70       240

    accuracy                           0.85       860
   macro avg       0.82      0.79      0.80       860
weighted avg       0.84      0.85      0.84       860



In [ ]:
from sklearn.metrics import classification_report
test_df_st_from_3_basic_models = pd.read_csv('test_df_st_from_3_basic_models_cross.csv', index_col=0)
test_df = pd.read_csv('olid-test.csv').drop(columns = ['labels'])
X_test = pd.concat([test_df_st_from_3_basic_models, test_df], axis = 1)
#X_test['base_lexicon_count'] = X_test['text'].apply(count_base_lexicon)
X_test['character_length_count'] = X_test['text'].apply(count_character_length)
X_test['word_count'] = X_test['text'].apply(count_text_length)
X_test['vader'] = X_test['text'].apply(get_vader)

X_test = X_test[['bert_predicted', 'hatebert_predicted','fbert_predicted',
                 'character_length_count', 'word_count','vader']]


#X_test['base_lexicon_count'] = transformer1.transform(X_test[['base_lexicon_count']])
X_test['character_length_count'] = transformer2.transform(X_test[['character_length_count']])
X_test['word_count'] = transformer3.transform(X_test[['word_count']])

meta_model_pred = meta_model.predict(X_test)

print(classification_report(test_df_st_from_3_basic_models['labels'],meta_model_pred))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87       620
           1       0.71      0.50      0.59       240

    accuracy                           0.80       860
   macro avg       0.77      0.71      0.73       860
weighted avg       0.79      0.80      0.79       860



<ipython-input-9-236f295469ea>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['character_length_count'] = transformer2.transform(X_test[['character_length_count']])
<ipython-input-9-236f295469ea>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['word_count'] = transformer3.transform(X_test[['word_count']])


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

cm = confusion_matrix(test_df_st_from_3_basic_models['labels'], meta_model_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

In [ ]:
test_df_st_from_3_basic_models